In [3]:
pip install music21

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 22.8 MB 110 kB/s eta 0:00:01
     |████████████████████████████████| 301 kB 15.0 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 7.6 MB/s eta 0:00:011
     |████████████████████████████████| 46 kB 8.7 MB/s  eta 0:00:01
     |████████████████████████████████| 5.3 MB 20 kB/s eta 0:00:0101
     |████████████████████████████████| 7.8 MB 130 kB/s eta 0:00:011
     |████████████████████████████████| 63 kB 4.5 MB/s eta 0:00:011
     |████████████████████████████████| 199 kB 10.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.1 MB 113 kB/s eta 0:00:011
     |████████████████████████████████| 249 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 2.8 MB 9.7 MB/s eta 0:00:01
     |████████████████████████████████| 107 kB 48.0 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 6.7 MB/s eta 0:00:01
     |██████████████████████████

In [3]:
from music21 import *

/Users/melodyqian/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
s = corpus.parse('bach/bwv65.2.xml')

In [6]:
s.analyze('key')

<music21.key.Key of a minor>

In [8]:
from music21 import corpus
mazurka=corpus.parse('chopin/mazurka06-2')

In [9]:
mazurka.analyze('key')

<music21.key.Key of c# minor>

In [20]:
configure.run()


______________________________________________________________________________ 
  
Welcome to the music21 Configuration Assistant. You will be guided through a 
number of questions to install and set up music21. Simply pressing return at a 
prompt will select a default, if available. 

You may run this configuration again at a later time by running 
music21/configure.py. 
  
______________________________________________________________________________ 
  
Defining an XML Reader permits automatically opening music21-generated 
MusicXML in an editor for display and manipulation when calling the show() 
method. Setting this option is highly recommended. 
  
[1] /Applications/Musescore4.app 
  
Choose a number from the preceding options (default is 1): 
MusicXML Reader set to: /Applications/Musescore4.app 
  
______________________________________________________________________________ 
  
The BSD-licensed music21 software is distributed with a corpus of encoded 
compositions which are 

In [21]:
nocturne=converter.parse('/Users/melodyqian/Documents/GitHub/pianocompetitionpredictions/test/chopnoc9.mxl', format='musicmxl')

In [24]:
us = environment.UserSettings()
us['musicxmlPath'] ='/Applications/Musescore4.app'

In [31]:
elements=nocturne.flat.notes

number_of_notes = 0
for elem in elements:
    if isinstance(elem, note.Note):
        number_of_notes += 1
    elif isinstance(elem, chord.Chord):
        number_of_notes += len(elem.pitches)



In [32]:
print(f'Total number of notes in the piece: {number_of_notes}')

Total number of notes in the piece: 1253
